In [1]:
import os
os.environ["MPLCONFIGDIR"] = "."
import gym
from manipulation_main.common import io_utils
import matplotlib.pyplot as plt
from ipycanvas import Canvas
import numpy as np
from PIL import Image

In [2]:
config = io_utils.load_yaml("config/gripper_grasp.yaml")
env = gym.make('gripper-env-v0', config=config)
done = False

dataset random_urdfs


In [3]:
class Button:
    def __init__(self, canvas, pt, size=50, text=""):
        self.canvas = canvas
        self.pt = pt
        self.size = 50
        canvas.fill_style = 'black'
        canvas.fill_rect(pt[0], pt[1], size)
        if text != "":
            canvas.font = '32px serif'
            canvas.fill_style = 'white'
            canvas.fill_text(text, pt[0], pt[1]+size)
    
    def is_clicked(self, pt):
        return self.pt[0] < pt[0] < self.pt[0] + self.size and \
            self.pt[1] < pt[1] < self.pt[1] + self.size



In [4]:
canvas = Canvas(width=500, height=500)
canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas.width, canvas.height)
north = Button(canvas, (50, 0), text="N")
south = Button(canvas, (50, 50), text="S")
west = Button(canvas, (0, 50), text="W")
east = Button(canvas, (100, 50), text="E")
up = Button(canvas, (175, 0), text="Up")
down = Button(canvas, (175, 50), text="Dn")
grab = Button(canvas, (250, 0), text="Gr")
release = Button(canvas, (250, 50), text="Rel")
buttons = [north, south, west, east, up, down, grab, release]

def handle_mouse_down_fn(x, y):
    global env, done, buttons, canvas
    bool_buttons = [z.is_clicked((x, y)) for z in buttons]
    n, s, w, e, u, d, g, rel = bool_buttons
    if done:
        obs = env.reset()
    action = np.zeros((5,), dtype=np.float)
    if (w): # left
        action[0] = 1.
    elif (e): # right
        action[0] = -1.
    elif (n): # up
        action[1] = 1.
    elif (s): # down
        action[1] = -1.
    elif (u): # approach
        action[2] = -1.
    elif (d): # deapproach
        action[2] = 1.
    elif (rel): # release ctrl
        action[4] = 1.
    elif (g): # grab shift
        action[4] = -1.

    obs, reward, done, _ = env.step(action)
    obs = obs.astype(np.uint8)
    obs = Image.fromarray(obs[..., :3])
    obs = obs.resize((obs.size[0] * 4, obs.size[1] * 4))
    canvas.put_image_data(np.array(obs), y=125)
    canvas.fill_style = 'white'
    canvas.fill_rect(0, 100, 500, 25)
    canvas.fill_style = 'black'
    canvas.font = '20px sans-serif'
    canvas.fill_text(f"reward: {reward}", 0, 125)


canvas.on_mouse_down(handle_mouse_down_fn)
canvas

Canvas(width=500)